# Check The Status of Model
Memeriksa status model yang telah dideploy pada server.

In [1]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/xss-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1'}]}


# Make Prediction Request

## Import Library
Mengimpor library yang dibutuhkan untuk membuat permintaan prediksi.

In [2]:
import requests
import tensorflow as tf
import base64

## Make functions for transform text
Membuat fungsi untuk mengubah teks menjadi format yang sesuai untuk input model.

In [3]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_text(text):
    example = tf.train.Example(features=tf.train.Features(feature={
        'Sentence': _bytes_feature(text)
        }))
    serialized_example = example.SerializeToString()
    return serialized_example

## Make function for predict
Mendefinisikan URL server model dan membuat fungsi untuk melakukan prediksi dengan menggunakan model yang telah dideploy.

In [4]:
model_server_url = 'http://localhost:8080/v1/models/xss-detection-model:predict'

def predict(text):
    example = serialize_text(text)
    json_data = {
      "signature_name":"serving_default",
      "instances":[
        {
          "examples":{"b64": base64.b64encode(example).decode('utf-8')}
        }
      ]
    }
    resp = requests.post(model_server_url, json=json_data)
    return resp.json()

## Test Predictions
Menyiapkan beberapa teks untuk diuji dan memanggil fungsi prediksi untuk mengevaluasi apakah teks tersebut terdeteksi sebagai XSS atau tidak.

In [5]:
texts = [b'	 </span> <span class=""reference-text"">', # not XSS
        b'<sup onkeypress=""alert(1)"" contenteditable>test</sup>'] # XSS
for text in texts:
    pred = predict(text)
    if pred['predictions'][0][0] > 0.5:
        print('Score Prediction :',pred['predictions'][0][0])
        print("XSS Detected\n")
    else:
        print('Score Prediction :',pred['predictions'][0][0])
        print("XSS Not Detected\n")


Score Prediction : 0.00526927365
XSS Not Detected

Score Prediction : 0.999993622
XSS Detected

